[Note]
*   Job: Load Sentiment fine tuned model to predict unknown data
*   Model:
> 1.   from folder "model_save_256FV2" etc
> 2.   pretrained by KccBertSentiment_256FV2.ipynb etc
*   Date: 2022.1.5

In [1]:
!pip install -q transformers
!pip install tqdm requests regex sentencepiece sacremoses

In [2]:
import torch

# avoid CUDA out of memory
torch.cuda.empty_cache()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("xpu")
    # print('We will use the TPU:', torch.cuda.get_device_name(0))

No GPU available, using the CPU instead.


In [3]:
from transformers import BertTokenizer
from IPython.display import clear_output

import numpy as np
from numpy import asarray 
from numpy import zeros
import matplotlib.pyplot as plt

import csv
import logging
import os
import pandas as pd

# 指定繁簡中文 BERT-BASE 預訓練模型
PRETRAINED_MODEL_NAME = "bert-base-chinese"  
# 取得此預訓練模型所使用的 tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
print("PyTorch 版本：", torch.__version__)

vocab = tokenizer.vocab
print("字典大小：", len(vocab))

PyTorch 版本： 1.11.0
字典大小： 21128


In [4]:
# 以Colab資料夾連結雲端硬碟
#!pip install google.colab #如未安裝取消註解後執行

#from google.colab import drive
#drive.mount('/content/drive')

#出現提示欄進行授權

#os.chdir('/content/drive//My Drive/KCC NLP 2021 Test/data') #切換目錄


In [5]:
"""
用來讀取訓練 / 測試集的 Dataset
Dataset 每次將 tsv 裡的一筆句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：句子的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
    
class SentimentDataset(Dataset):
    # 讀取預處理的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        # assert mode in ["KccTrain", "KccVerf", "KccTest"] 定義訓練,驗證,測試模式
        assert mode in ["LYVerf-Senti256FV2", "10th_green_ind_512"] #前面＿LYVerf-Senti256FV2作為有答案的驗證檔，後面放未標記想要測試的檔案
        self.mode = mode
        self.df = pd.read_csv(mode + ".csv", sep="\t").fillna("")
        self.len = len(self.df)
        self.tokenizer = tokenizer  # 使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        if self.mode == "10th_green_ind_512": #這邊先忽略未標記的data
            text = self.df.iloc[idx,0]
            label_tensor = None
        else:
            text = self.df.iloc[idx,1]
            label_tensor = None
            
        # 建立句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokenTx = self.tokenizer.tokenize(text)
        word_pieces += tokenTx + ["[SEP]"]
        lenTx = len(word_pieces)
                
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將包含 [SEP] 的 token 對映位置均設為 0
        segments_tensor = torch.tensor([0] * lenTx, dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        print("class SentimentDataset(Dataset) return len(self.df) = ",len(self.df))
        return self.len  

In [6]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 載入前面定義的SentimentDataset，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

# 本函式的輸入 `samples` 為list，list中每個 element 都是
# SentimentDataset 回傳的一個 dataset ，每個dataset樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 本函式會對前兩個 tensors 作 zero padding，並產生 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列最長長度
    tokens_tensors = pad_sequence(tokens_tensors,batch_first=True)
    segments_tensors = pad_sequence(segments_tensors,batch_first=True)
        
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape,dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

In [7]:
from transformers import BertForSequenceClassification
# fine tuned model path:
output_dir = '/Users/jojoteng/Documents/10th_model/green' #儲存的目的地 MODEL &TOKENZER

# https://huggingface.co/docs/transformers/main_classes/model
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

In [8]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                      token_type_ids=segments_tensors, 
                      attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions


In [9]:
#下面使用未標記的來驗證

In [10]:
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
tsSet = SentimentDataset("10th_green_ind_512", tokenizer=tokenizer) #測試未標記的檔案(KCCTEST)
tsLoader = DataLoader(tsSet, batch_size=16, collate_fn=create_mini_batch)
print(tsSet.df.head(10))

# 用分類模型預測測試集
predicTest = get_predictions(model, tsLoader)

df = pd.DataFrame({"Predict": predicTest.tolist()})
print(df.head(20))

df_pred = pd.concat([tsSet.df.loc[:, ["Text"]], df.loc[:, 'Predict']], axis=1)

df_pred.to_csv('10th_green_ind_512_predict.csv', index=False, encoding='utf_8_sig')#經過 MODEL 標記後儲存成檔案
df_pred.head(20)

                                                Text
0  南迴醫療再升級大武緊急救護中心正式啟動台東縣偏鄉的南迴公路上一旦發生事故需要緊急救護時往北5...
1  今天是2020年第一天也是21世紀第三個十年的開始迎接新時代我要提出伍麗華的台灣新藍圖🔺第一...
2  什麼叫做族群主流化其實道理很簡單就是社會上的不同族群都可以互相尊重互相學習互相欣賞這樣台灣一...
3  什麼叫做族群主流化其實道理很簡單就是社會上的不同族群都可以互相尊重互相學習互相欣賞這樣台灣一...
4  早安倒數九天跟大家分享滑球王子阿福陽建福給我的勉勵^^陽建福跟我說2008年的奧運棒球資格賽...
5  ⑨號陳瑩-捍衛原住民土地權益的守護者土地對一般人來說只是種米種菜蓋房子蓋大樓但土地對原住民來...
6  今天有件不幸的消息：國軍直升機發生事故包括沈一鳴參謀總長在內共有8人罹難請大家一起為國軍禱告...
7  悲傷的一天今天有好幾位優秀的國軍將領與軍官弟兄因公殉職我們安定的生活全倚靠國軍官兵不分晝夜辛...
8  我們推薦伍麗華鄒族歌手的推薦高蕾雅／歌手主持人及演員來自達邦部落認識麗華姐是因為泰武國小知道...
9  最近好多人跟我說：處長妳在兩次的政見發表會講得真好我也意外發現芋傳媒完整報導我在中選會公辦政...
class SentimentDataset(Dataset) return len(self.df) =  1082
    Predict
0         1
1         0
2         1
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19        1


,Text,Predict
0,南迴醫療再升級大武緊急救護中心正式啟動台東縣偏鄉的南迴公路上一旦發生事故需要緊急救護時往北5...,1
1,今天是2020年第一天也是21世紀第三個十年的開始迎接新時代我要提出伍麗華的台灣新藍圖🔺第一...,0
2,什麼叫做族群主流化其實道理很簡單就是社會上的不同族群都可以互相尊重互相學習互相欣賞這樣台灣一...,1
3,什麼叫做族群主流化其實道理很簡單就是社會上的不同族群都可以互相尊重互相學習互相欣賞這樣台灣一...,1
4,早安倒數九天跟大家分享滑球王子阿福陽建福給我的勉勵^^陽建福跟我說2008年的奧運棒球資格賽...,1
5,⑨號陳瑩-捍衛原住民土地權益的守護者土地對一般人來說只是種米種菜蓋房子蓋大樓但土地對原住民來...,1
6,今天有件不幸的消息：國軍直升機發生事故包括沈一鳴參謀總長在內共有8人罹難請大家一起為國軍禱告...,1
7,悲傷的一天今天有好幾位優秀的國軍將領與軍官弟兄因公殉職我們安定的生活全倚靠國軍官兵不分晝夜辛...,1
8,我們推薦伍麗華鄒族歌手的推薦高蕾雅／歌手主持人及演員來自達邦部落認識麗華姐是因為泰武國小知道...,1
9,最近好多人跟我說：處長妳在兩次的政見發表會講得真好我也意外發現芋傳媒完整報導我在中選會公辦政...,1


In [11]:
#第二層

In [12]:
#第二層-1
"""
用來讀取訓練 / 測試集的 Dataset
Dataset 每次將 tsv 裡的一筆句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：句子的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
    
class SentimentDataset(Dataset):
    # 讀取預處理的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        # assert mode in ["KccTrain", "KccVerf", "KccTest"] 定義訓練,驗證,測試模式
        assert mode in ["LYVerf-Senti256FV2", "2021_green_512"] #前面＿KccVerf-Senti256FV2作為有答案的驗證檔，後面放未標記想要測試的檔案
        self.mode = mode
        self.df = pd.read_csv(mode + ".csv", sep="\t").fillna("")
        self.len = len(self.df)
        self.tokenizer = tokenizer  # 使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        if self.mode == "2021_green_512": #這邊先忽略未標記的data
            text = self.df.iloc[idx,0]
            label_tensor = None
        else:
            text = self.df.iloc[idx,1]
            label_tensor = None
            
        # 建立句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokenTx = self.tokenizer.tokenize(text)
        word_pieces += tokenTx + ["[SEP]"]
        lenTx = len(word_pieces)
                
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將包含 [SEP] 的 token 對映位置均設為 0
        segments_tensor = torch.tensor([0] * lenTx, dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        print("class SentimentDataset(Dataset) return len(self.df) = ",len(self.df))
        return self.len  

In [13]:
#第二層-2
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 載入前面定義的SentimentDataset，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

# 本函式的輸入 `samples` 為list，list中每個 element 都是
# SentimentDataset 回傳的一個 dataset ，每個dataset樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 本函式會對前兩個 tensors 作 zero padding，並產生 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列最長長度
    tokens_tensors = pad_sequence(tokens_tensors,batch_first=True)
    segments_tensors = pad_sequence(segments_tensors,batch_first=True)
        
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape,dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

In [14]:
#第二層-3
from transformers import BertForSequenceClassification
# fine tuned model path:
output_dir = '/Users/jojoteng/Documents/10th_model/green' #儲存的目的地 MODEL &TOKENZER

# https://huggingface.co/docs/transformers/main_classes/model
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

In [15]:
#第二層-4
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                      token_type_ids=segments_tensors, 
                      attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions

In [ ]:
#第二層-5
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
tsSet = SentimentDataset("2021_green_512", tokenizer=tokenizer) #測試未標記的檔案
tsLoader = DataLoader(tsSet, batch_size=16, collate_fn=create_mini_batch)
print(tsSet.df.head(10))

# 用分類模型預測測試集
predicTest = get_predictions(model, tsLoader)

df = pd.DataFrame({"Predict": predicTest.tolist()})
print(df.head(20))

df_pred = pd.concat([tsSet.df.loc[:, ["Text"]], df.loc[:, 'Predict']], axis=1)

df_pred.to_csv('2021_green_512_predict.csv', index=False, encoding='utf_8_sig')#經過 MODEL 標記後儲存成檔案
df_pred.head(20)

                                                Text
0  感謝阿龍林佳龍來大林陪我掃街沿途有許多民眾給我們熱情的支持真的非常感謝雖然我們從阿里山下來已...
1  2020台灣要贏嘉義人站出來明天是2020開工的第一天阿文將會出現在大士爺廟前與我們山區各鄉...
2  早安我們在嘉義阿里山日出印象音樂會迎接第一道曙光的到來日出印象音樂會是我當縣長時創立的今年已...
3  2020台灣要贏嘉義人站出來今天是2020開工的第一天阿文現在在大士爺廟前跟大家問候我們其他...
4             對於黑鷹直升機上午重大傷亡事件國軍弟兄罹難阿文跟全體競選團隊表達最深沉的哀痛
5  大家還記得去年蔡總統來嘉義視察中埔嗎當時我們規劃的公館滯洪池總面積約12公頃滯洪量52萬立方...
6  國軍黑鷹事件正駕駛葉上校是我們中埔的鄉親也擔任我們中埔學校的家長委員他在這一次事件中英勇犧牲...
7  一起為國家祈福為大家求平安🙏今天早上我們前往竹崎獨立山奉天岩向觀音佛祖祈求國泰民安大家能平安...
8  大嘉義之夜🔴台灣隊長趕緊集結我們現在在嘉義市立棒球場就是為著自己人挺自己人選前倒數⑤天阿文和...
9  剛剛我和游錫堃院長一起到新港向各位鄉親打招呼游院長從當院長的時候就很支持阿文總是給我鼓勵這次...
class SentimentDataset(Dataset) return len(self.df) =  21759
    Predict
0         1
1         1
2         1
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        0
15        1
16        1
17        1
18        1
19        1


In [22]:
#第三層

In [23]:
#第三層-1
"""
用來讀取訓練 / 測試集的 Dataset
Dataset 每次將 tsv 裡的一筆句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：句子的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
    
class SentimentDataset(Dataset):
    # 讀取預處理的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer):
        # assert mode in ["KccTrain", "KccVerf", "KccTest"] 定義訓練,驗證,測試模式
        assert mode in ["LYVerf-Senti256FV2", "F_2021_green_3"] #前面＿KccVerf-Senti256FV2作為有答案的驗證檔，後面放未標記想要測試的檔案
        self.mode = mode
        self.df = pd.read_csv(mode + ".csv", sep="\t").fillna("")
        self.len = len(self.df)
        self.tokenizer = tokenizer  # 使用 BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        if self.mode == "F_2021_green_3": #這邊先忽略未標記的data
            text = self.df.iloc[idx,0]
            label_tensor = None
        else:
            text = self.df.iloc[idx,1]
            label_tensor = None
            
        # 建立句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokenTx = self.tokenizer.tokenize(text)
        word_pieces += tokenTx + ["[SEP]"]
        lenTx = len(word_pieces)
                
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將包含 [SEP] 的 token 對映位置均設為 0
        segments_tensor = torch.tensor([0] * lenTx, dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor)
    
    def __len__(self):
        print("class SentimentDataset(Dataset) return len(self.df) = ",len(self.df))
        return self.len  

In [24]:
#地三層-2
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 載入前面定義的SentimentDataset，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

# 本函式的輸入 `samples` 為list，list中每個 element 都是
# SentimentDataset 回傳的一個 dataset ，每個dataset樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 本函式會對前兩個 tensors 作 zero padding，並產生 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列最長長度
    tokens_tensors = pad_sequence(tokens_tensors,batch_first=True)
    segments_tensors = pad_sequence(segments_tensors,batch_first=True)
        
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape,dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

In [25]:
#第三層-3
from transformers import BertForSequenceClassification
# fine tuned model path:
output_dir = '/Users/jojoteng/Documents/10th_model' #儲存的目的地 MODEL &TOKENZER

# https://huggingface.co/docs/transformers/main_classes/model
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

In [26]:
#第三層-4
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                      token_type_ids=segments_tensors, 
                      attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions

In [27]:
#第三層-5
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
tsSet = SentimentDataset("F_2021_green_3", tokenizer=tokenizer) #測試未標記的檔案(KCCTEST)
tsLoader = DataLoader(tsSet, batch_size=16, collate_fn=create_mini_batch)
print(tsSet.df.head(10))

# 用分類模型預測測試集
predicTest = get_predictions(model, tsLoader)

df = pd.DataFrame({"Predict": predicTest.tolist()})
print(df.head(20))

df_pred = pd.concat([tsSet.df.loc[:, ["Text"]], df.loc[:, 'Predict']], axis=1)

df_pred.to_csv('F_2021_green_3_predict.csv', index=False, encoding='utf_8_sig')#經過 MODEL 標記後儲存成檔案
df_pred.head(20)

                                                Text
0  衛環委員會安排桃園藻礁考察惠員聽完簡報一路也與經濟部曾文生次長環保署張子敬署長繼續討論大潭三...
1  今天上午易餘與高嘉瑜立法委員江永昌高雄小金剛許智傑何志偉委員召開『性別暴力零容忍!誰作被害人...
2  晚安聖誕快樂今天假日活動滿滿晚上瑞隆來到大林蒲參加110年大林蒲海風音樂會活動感謝小港區公所...
3  遲來了但我們做到了國營事業優秀運動員可接受商業代言還記得今年東京奧運王齊麟李洋帶給我們的感動...
4  盧秀燕不在意三天後的公投結果在意的是明年可能出現的政治危機繼侯友宜的千字文表態不表態後今天盧...
5  民眾來看了幾次書法展現場寫下了他的觀察評論雖然過獎但想分享觀黃國書先生書法創作展兼擅各式書體...
6  顏家的正義就是只有我可以為所欲為的在地正義中二選情進入白熱化顏家的黑歷史也一件一件被翻出其中...
7  四個不同意台灣更有力我們今天持續向鄉親們宣傳我們的理念一項一項說明1218的公投到底要怎麼投...
8  朝野立場難免分合立法院正副院長處理議事時依法必須維持議事中立任何提案只要符合程序都應該被公平...
9  感謝鄉親關心滿滿在地成績報您知昨天晚上我參與了好朋友市議員劉耀仁舉辦的公投說明會現場無論是楊...
class SentimentDataset(Dataset) return len(self.df) =  1959
    Predict
0         1
1         1
2         0
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19        1


,Text,Predict
0,衛環委員會安排桃園藻礁考察惠員聽完簡報一路也與經濟部曾文生次長環保署張子敬署長繼續討論大潭三...,1
1,今天上午易餘與高嘉瑜立法委員江永昌高雄小金剛許智傑何志偉委員召開『性別暴力零容忍!誰作被害人...,1
2,晚安聖誕快樂今天假日活動滿滿晚上瑞隆來到大林蒲參加110年大林蒲海風音樂會活動感謝小港區公所...,0
3,遲來了但我們做到了國營事業優秀運動員可接受商業代言還記得今年東京奧運王齊麟李洋帶給我們的感動...,1
4,盧秀燕不在意三天後的公投結果在意的是明年可能出現的政治危機繼侯友宜的千字文表態不表態後今天盧...,1
5,民眾來看了幾次書法展現場寫下了他的觀察評論雖然過獎但想分享觀黃國書先生書法創作展兼擅各式書體...,1
6,顏家的正義就是只有我可以為所欲為的在地正義中二選情進入白熱化顏家的黑歷史也一件一件被翻出其中...,1
7,四個不同意台灣更有力我們今天持續向鄉親們宣傳我們的理念一項一項說明1218的公投到底要怎麼投...,1
8,朝野立場難免分合立法院正副院長處理議事時依法必須維持議事中立任何提案只要符合程序都應該被公平...,1
9,感謝鄉親關心滿滿在地成績報您知昨天晚上我參與了好朋友市議員劉耀仁舉辦的公投說明會現場無論是楊...,1


In [122]:
#下面使用已標記的來驗證

In [123]:
# 建立驗證集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
verSet = SentimentDataset("LYVerf-Senti256FV2", tokenizer=tokenizer)
verLoader = DataLoader(verSet, batch_size= 32, collate_fn=create_mini_batch)
print(verSet.df.head(10))

print("-------------------------用分類模型預測驗證集-------------------------")
predictions = get_predictions(model, verLoader)

df = pd.DataFrame({"Predict": predictions.tolist()})
print(df.head(10))

df_pred = pd.concat([verSet.df.loc[:, ["Text","Label"]], df.loc[:, 'Predict']], axis=1)

idnoList = []
ckList = []
ckCunt = 0
for i in range(0,len(df_pred)):
   ck = ""
   if df_pred.at[i,"Label"] != df_pred.at[i,"Predict"]:
      ck = "Check"
      ckCunt += 1
   idnoList.append(i + 1)
   ckList.append(ck)

pRate = (1 - round(ckCunt/i,3)) * 100
strEstm = "總驗證筆數= "+str(i)+" 不符筆數= "+str(ckCunt)+" 精確度= "+str(pRate)
print(strEstm)
# print("總驗證筆數= ",i+1," 不符筆數= ", ckCunt," 不符比率= ",ckCunt/(i+1))

df_pred.insert(3, column="Check", value=ckList) #驗證不符的話就標上CHECK
df_pred.insert(0, column="SeqNo", value=idnoList)
df_Predict = df_pred.append({
    "SeqNo": 0,
    "Text": strEstm,
    "Label": "0",
    "Predict": "0",
    "Check": ""
}, ignore_index=True)

df_Predict.to_csv('LYVerf256FV2__predCk512.csv', index=False, encoding='utf_8_sig') #儲存驗證完的檔案
df_Predict.head(6)
df_Predict.tail(6)

   Label                                               Text
0      1             今天的喜宴行程特別多祝福每一對攜手同心也期待每一對都能永遠幸福然後貝比生很多
1      1  今日參加沙崙綠能科學城啟動儀式由林全院長親臨主持俊憲也與賴清德市長一同出席參加綠能科技產業推...
2      0  經濟部掌管的非營業基金高達四百多億這些基金的設置都有其特殊目的不同基金也有不同用途但是共同的...
3      1  花蓮南北狹長超過一百公里醫療的照護跟資源都非常艱困謝謝門諾照顧新生兒的醫護團隊不斷守護著最小...
4      1  20142016年美國對台灣施壓的276品項的農藥開放馬政府時代已經開放了152項有的還趕在...
5      1  服務時間暫停通知親愛的鄉親您好本週六1230因適逢連續假日俊憲服務時間暫停一次平日若有需要服...
6      1                        LIVE民進黨立法院黨團人權普世價值中國立即放人記者會
7      1  素美今天前往楊梅故事館參觀我畫故我在古明朗羅汶淇雙人聯展這場畫展是80歲古明朗阿公人生中的第...
8      1  見證台灣首位羽球世界球后國書邀請大家一起為戴資穎加油在這禮拜的香港羽球超級系列賽中戴資穎Ta...
9      0  航太新指標無人機國造國家海巡單位的業務包山包海業務成本與人力風險始終居高不下然而無人飛行科技...
-------------------------用分類模型預測驗證集-------------------------
class SentimentDataset(Dataset) return len(self.df) =  293
   Predict
0        1
1        1
2        0
3        1
4        1
5        1
6        1
7        1
8        1
9        0
總驗證筆數= 292 不符筆數= 32 精確度= 89.0


,SeqNo,Text,Label,Predict,Check
288,289,推動黑潮發電落實能綠能科技政策黑潮又稱日本暖流是太平洋洋流的一環為全球第二大洋流只居於墨西哥...,1,1,
289,290,第N天的甲級動員最期待的就是每天的早餐吃飽才有力氣繼續衝換穿狗狗衣服換個心情再戰前瞻感謝立法...,1,1,
290,291,政府應輔導不應與民爭利歐珀今日於交通委員會針對行政院公共工程委員會主委吳宏謀進行質詢歐珀首先...,0,0,
291,292,謝謝志工伙伴為社會的付出與努力106年生命線全國年會聚集台東生命飛揚,1,1,
292,293,昨晚外交國防委員會挑燈夜戰終於審完了2018年度國防部預算至此這個會期本委員會的預算案審查完...,0,0,
293,0,總驗證筆數= 292 不符筆數= 32 精確度= 89.0,0,0,


In [124]:
#下面開始檢證未標記的data

In [149]:
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
tsSet = SentimentDataset("F_2016_green_1", tokenizer=tokenizer) #測試未標記的檔案(KCCTEST)
tsLoader = DataLoader(tsSet, batch_size=16, collate_fn=create_mini_batch)
print(tsSet.df.head(10))

# 用分類模型預測測試集
predicTest = get_predictions(model, tsLoader)

df = pd.DataFrame({"Predict": predicTest.tolist()})
print(df.head(20))

df_pred = pd.concat([tsSet.df.loc[:, ["Text"]], df.loc[:, 'Predict']], axis=1)

df_pred.to_csv('F_2016_green_1_predict.csv', index=False, encoding='utf_8_sig')#經過 MODEL 標記後儲存成檔案
df_pred.head(20)

                                                Text
0  今天是2016年的第一天劉櫂豪服務團隊早上準時參加升旗典禮以嶄新與團結的心情迎接新的開始但在...
1  老人福利政策2016年的第一天易餘和民進黨不分區立委提名第二名的吳玉琴一同前往布袋新塭老人食...
2  發呆日一起學著過生活難得的連續假期大家都出去玩了呢🚲還是盡情待在家享受悠閒的時光呢🏡彰化今年...
3  開放美豬爭議當選舉抹黑恐嚇牌吳育仁委員針對蔡英文主席開放美豬爭議質疑蔡英文當選後就要開放含有...
4  20160101民眾日報第B10版高雄政經蚵子寮漁港改善邱志偉爭經費本報記者宋正忠高雄報導保...
5  今天是2016年的第一天孫綾今天一早就與Q版小英小豬胖卡車隊一起從台灣頭石門出發跟大家問好沿...
6  今天是2016年元旦很多新制上路2013開始也有一項新制是我推動實施的就是車齡10年以上小汽...
7  坊間一直有北慶忠中汝芬的說法從這財產申報資料看來果然民不虛傳立法委員在其任內土地從20筆暴增...
8  旺旺中國集團動用旗下媒體圍剿黃國昌這件事真的不談談不行媒體可以有立場政治人物可以被檢驗但是像...
9  點亮台灣照亮嘉義鹿草鄉影片右下角記得點選高畫質HD影像觀看歡迎分享鴨母寮大排是鹿草鄉的心臟貫...
class SentimentDataset(Dataset) return len(self.df) =  9999
    Predict
0         1
1         1
2         1
3         1
4         1
5         1
6         1
7         1
8         1
9         1
10        1
11        1
12        1
13        1
14        1
15        1
16        1
17        1
18        1
19        1


,Text,Predict
0,今天是2016年的第一天劉櫂豪服務團隊早上準時參加升旗典禮以嶄新與團結的心情迎接新的開始但在...,1
1,老人福利政策2016年的第一天易餘和民進黨不分區立委提名第二名的吳玉琴一同前往布袋新塭老人食...,1
2,發呆日一起學著過生活難得的連續假期大家都出去玩了呢🚲還是盡情待在家享受悠閒的時光呢🏡彰化今年...,1
3,開放美豬爭議當選舉抹黑恐嚇牌吳育仁委員針對蔡英文主席開放美豬爭議質疑蔡英文當選後就要開放含有...,1
4,20160101民眾日報第B10版高雄政經蚵子寮漁港改善邱志偉爭經費本報記者宋正忠高雄報導保...,1
5,今天是2016年的第一天孫綾今天一早就與Q版小英小豬胖卡車隊一起從台灣頭石門出發跟大家問好沿...,1
6,今天是2016年元旦很多新制上路2013開始也有一項新制是我推動實施的就是車齡10年以上小汽...,1
7,坊間一直有北慶忠中汝芬的說法從這財產申報資料看來果然民不虛傳立法委員在其任內土地從20筆暴增...,1
8,旺旺中國集團動用旗下媒體圍剿黃國昌這件事真的不談談不行媒體可以有立場政治人物可以被檢驗但是像...,1
9,點亮台灣照亮嘉義鹿草鄉影片右下角記得點選高畫質HD影像觀看歡迎分享鴨母寮大排是鹿草鄉的心臟貫...,1


In [ ]:
#可更換其需預測之檔案

In [69]:
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
tsSet = SentimentDataset("9", tokenizer=tokenizer) #測試未標記的檔案(KCCTEST)
tsLoader = DataLoader(tsSet, batch_size=16, collate_fn=create_mini_batch)
print(tsSet.df.head(10))

# 用分類模型預測測試集
predicTest = get_predictions(model, tsLoader)

df = pd.DataFrame({"Predict": predicTest.tolist()})
print(df.head(10))

df_pred = pd.concat([tsSet.df.loc[:, ["date","Text"]], df.loc[:, 'Predict']], axis=1)

df_pred.to_csv('LYVerf256FV2_predict_5129th_green_test.csv', index=False, encoding='utf_8_sig')#將KCCTEST經過 MODEL 標記後儲存成檔案
df_pred.head(10)

FileNotFoundError: [Errno 2] No such file or directory: '9th_green_test.csv'

In [66]:
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
tsSet = SentimentDataset("512_green_2th", tokenizer=tokenizer) #測試未標記的檔案(KCCTEST)
tsLoader = DataLoader(tsSet, batch_size=16, collate_fn=create_mini_batch)
print(tsSet.df.head(10))

# 用分類模型預測測試集
predicTest = get_predictions(model, tsLoader)

df = pd.DataFrame({"Predict": predicTest.tolist()})
print(df.head(10))

df_pred = pd.concat([tsSet.df.loc[:, ["Text"]], df.loc[:, 'Predict']], axis=1)

df_pred.to_csv('LYVerf256FV2_predict_512_green_2th.csv', index=False, encoding='utf_8_sig')#將KCCTEST經過 MODEL 標記後儲存成檔案
df_pred.head(10)

                                                Text
0  RCEP簽署，台灣不須妄自菲薄本週末最大的經貿新聞，就是醞釀多年的RCEP正式簽署，時值美國...
1  曹公234週年聖誕登場立委邱志偉獲曹公奬2020/11/01〔記者陳文嬋／高雄報導〕曹公23...
2  今天我必須向原能會謝曉星主委質詢核廢料從核電廠移除、提升原能會員工協助方案這兩個議題。核廢何...
3  校園零萊豬！思瑤一路緊盯營養午餐3368校換約💯％完成！這幾天，思瑤每一天跟教育部潘部長、國...
4  南鐵東移案在制度面上除了立院同仁范雲跟洪申翰委員針對制度的改善方案外，我同時要求警政署應該針...
5  認真負責的蘇貞昌院長已積極處理，交通部需繃緊神經！此次猴硐瑞芳之間，列管C級邊坡滑動，幸好司...
6  地震搖的蠻久，大家還好嗎？報一下平安吧！行政院災害防救辦公室21：40初報10日21:19宜...
7  今天立法院衛環委員會，我針對提升自殺防治專線量能與減輕照護住院病患負擔兩項議題做出質詢：提升...
8  劉櫂豪爭取450萬元協助大王國中新建棒球場促進體育發展劉櫂豪委員協助大王國中向教育部體育署爭...
9  全國首座韌性都市林真的來到中和四號公園了！永昌前天到四號公園，與中和社大志工一起種下19種台...
class SentimentDataset(Dataset) return len(self.df) =  6735
   Predict
0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1


,Text,Predict
0,RCEP簽署，台灣不須妄自菲薄本週末最大的經貿新聞，就是醞釀多年的RCEP正式簽署，時值美國...,1
1,曹公234週年聖誕登場立委邱志偉獲曹公奬2020/11/01〔記者陳文嬋／高雄報導〕曹公23...,1
2,今天我必須向原能會謝曉星主委質詢核廢料從核電廠移除、提升原能會員工協助方案這兩個議題。核廢何...,1
3,校園零萊豬！思瑤一路緊盯營養午餐3368校換約💯％完成！這幾天，思瑤每一天跟教育部潘部長、國...,1
4,南鐵東移案在制度面上除了立院同仁范雲跟洪申翰委員針對制度的改善方案外，我同時要求警政署應該針...,1
5,認真負責的蘇貞昌院長已積極處理，交通部需繃緊神經！此次猴硐瑞芳之間，列管C級邊坡滑動，幸好司...,1
6,地震搖的蠻久，大家還好嗎？報一下平安吧！行政院災害防救辦公室21：40初報10日21:19宜...,1
7,今天立法院衛環委員會，我針對提升自殺防治專線量能與減輕照護住院病患負擔兩項議題做出質詢：提升...,1
8,劉櫂豪爭取450萬元協助大王國中新建棒球場促進體育發展劉櫂豪委員協助大王國中向教育部體育署爭...,1
9,全國首座韌性都市林真的來到中和四號公園了！永昌前天到四號公園，與中和社大志工一起種下19種台...,1


In [45]:
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
tsSet = SentimentDataset("512_tsp_2th", tokenizer=tokenizer) #測試未標記的檔案(KCCTEST)
tsLoader = DataLoader(tsSet, batch_size=16, collate_fn=create_mini_batch)
print(tsSet.df.head(10))

# 用分類模型預測測試集
predicTest = get_predictions(model, tsLoader)

df = pd.DataFrame({"Predict": predicTest.tolist()})
print(df.head(10))

df_pred = pd.concat([tsSet.df.loc[:, ["Text"]], df.loc[:, 'Predict']], axis=1)

df_pred.to_csv('LYVerf256FV2_predict_512_tsp_2th.csv', index=False, encoding='utf_8_sig')#將KCCTEST經過 MODEL 標記後儲存成檔案
df_pred.head(10)

                                                Text
0  3Q日誌這週末跑了許多行程，沙鹿親子藝文季、大肚永順國小校慶、龍井龍港國小校慶暨新校舍落成啟...
1  週末做什麼？還沒想到嗎？本週六有 3Q文化祭蕭建平電視木偶劇團 的光雕布袋戲演出在霧峰哦白天...
2  台灣是台灣人的台灣我常講，我希望台派能夠最大化，在最大敵人消滅前，能砲口一致對外，守護台灣基...
3  ⭐⭐3Q應援！中秋團圓『扇』起來！⭐⭐ 即日起，只要到3Q陳柏惟台中的五個服務處，出示goo...
4  我的得獎致詞就寫在這了。謝謝公督盟、也謝謝團隊大家一起努力，最重要的是，謝謝台中人給我這個機...
5  口罩、酒精、額溫槍，手收洗起來！台灣加油 口罩是最好的平安符253天零本土後，台灣出現本土案...
6  國家級防災演練來襲！今天是9月21日，是台灣人集體記憶裡很深刻的日子，同時也是全國防災日。內...
7  最近3Q壘球隊的友誼賽交流邀約越來越多，成立的初心是我一直認為棒壘球是擁有台灣精神，可大力發...
8  玻璃骰子碎滿地本日媒體詢問，中美較勁，雙方不斷秀肌肉，怎麼看台灣在國防軍事上的佈局？我是這樣...
9  今日是冬至，吃過湯圓了嗎？喜歡鹹的還是甜的？有句傳說是：冬至黑，過年疏；冬至疏，過年黑說冬至...
class SentimentDataset(Dataset) return len(self.df) =  133
   Predict
0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1


,Text,Predict
0,3Q日誌這週末跑了許多行程，沙鹿親子藝文季、大肚永順國小校慶、龍井龍港國小校慶暨新校舍落成啟...,1
1,週末做什麼？還沒想到嗎？本週六有 3Q文化祭蕭建平電視木偶劇團 的光雕布袋戲演出在霧峰哦白天...,1
2,台灣是台灣人的台灣我常講，我希望台派能夠最大化，在最大敵人消滅前，能砲口一致對外，守護台灣基...,1
3,⭐⭐3Q應援！中秋團圓『扇』起來！⭐⭐ 即日起，只要到3Q陳柏惟台中的五個服務處，出示goo...,1
4,我的得獎致詞就寫在這了。謝謝公督盟、也謝謝團隊大家一起努力，最重要的是，謝謝台中人給我這個機...,1
5,口罩、酒精、額溫槍，手收洗起來！台灣加油 口罩是最好的平安符253天零本土後，台灣出現本土案...,1
6,國家級防災演練來襲！今天是9月21日，是台灣人集體記憶裡很深刻的日子，同時也是全國防災日。內...,1
7,最近3Q壘球隊的友誼賽交流邀約越來越多，成立的初心是我一直認為棒壘球是擁有台灣精神，可大力發...,1
8,玻璃骰子碎滿地本日媒體詢問，中美較勁，雙方不斷秀肌肉，怎麼看台灣在國防軍事上的佈局？我是這樣...,1
9,今日是冬至，吃過湯圓了嗎？喜歡鹹的還是甜的？有句傳說是：冬至黑，過年疏；冬至疏，過年黑說冬至...,1
